# Machine Learning

For the machine learning portion of the project, we decided to use `KNeighborsRegressor`, `RandomForestRegressor`, and `RegressionEnsembler` models to see which combination of features and parameters would yield the best test and validation error.

In [1]:
# Import statements
%matplotlib inline
import numpy as np
import pandas as pd
import itertools
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import warnings; warnings.simplefilter('ignore')

After reading in the dataframe created from the first notebook, we split it into training and test dataframes. The test dataframe that we would predict on had player information for the `2017` season, and the training dataframe has player information from `2012 - 2016`

In [2]:
# Create the training dataset (2008 - 2017)
final_df = pd.read_csv("/home/jovyan/Data301Winter2019/project/player.csv")
player_df = final_df[final_df["season"] != 2017].reset_index().drop(columns=["index"])
test_df = final_df[final_df["season"] == 2017].reset_index().drop(columns=["index"])
player_df.head()

,name,season,age,team_id,pos,g,gs,mp_per_g,fg_pct,fg3_per_g,...,pf_per_g,pts_per_g,height,weight,wins,losses,rank_team,division,conference,finish
0,Alex Abrines,2016.0,23.0,OKC,SG,68.0,6.0,15.5,0.393,1.4,...,1.7,6.0,78,200,47.0,35.0,2.0,Northwest,W,6.0
1,Steven Adams,2016.0,23.0,OKC,C,80.0,80.0,29.9,0.571,0.0,...,2.4,11.3,84,265,47.0,35.0,2.0,Northwest,W,6.0
2,Semaj Christon,2016.0,24.0,OKC,PG,64.0,1.0,15.2,0.345,0.2,...,1.2,2.9,75,190,47.0,35.0,2.0,Northwest,W,6.0
3,Norris Cole,2016.0,28.0,OKC,PG,13.0,0.0,9.6,0.308,0.2,...,1.4,3.3,74,175,47.0,35.0,2.0,Northwest,W,6.0
4,Nick Collison,2016.0,36.0,OKC,PF,20.0,0.0,6.4,0.609,0.0,...,0.9,1.7,82,255,47.0,35.0,2.0,Northwest,W,6.0


### Model and feature selection:

In [3]:
# Features that are correlated to average points per game
features = ["name", "height", "weight", "pos", "g", "mp_per_g", "efg_pct", "ft_pct"]

# creating a power set of all possible feature combinations
list_of_features = []
for n in range(1, len(features)+1):
    combination = [list(i) for i in itertools.combinations(features, n)]
    list_of_features.extend(combination)

In [4]:
y = player_df["pts_per_g"]
vec = DictVectorizer(sparse=False)
scaler = StandardScaler()

In [5]:
# Function that determines the best combination of features
def find_best_features(model, list_of_features, y):
    # Set up the pipeline
    pipeline = Pipeline([("vectorizer", vec), ("scaler", scaler), ("fit", model)])
    
    # Create a dictionary of features and its respective rmse value
    dictionary = {}
    for features in list_of_features:
        X_dict = player_df[features].to_dict(orient="records")
        scores = cross_val_score(pipeline, X_dict, y, cv=10, scoring="neg_mean_squared_error")
        rmse = np.sqrt(np.mean(-scores))
        dictionary.update({str(features):rmse})
    return dictionary

In [ ]:
# Evaluate the model with the best set of features and lowest RMSE
list_of_models = [RandomForestRegressor(n_estimators = 10), 
                  KNeighborsRegressor(n_neighbors = 10)]
for model in list_of_models:
    dictionary = find_best_features(model, list_of_features, y)
    min_rmse = min(dictionary.items(), key=lambda x: x[1]) 
    print("Model: " + str(model))
    print("Best features: " + min_rmse[0])
    print("RMSE value: " + str(min_rmse[1]) + "\n")

Model: RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
Best features: ['name', 'height', 'weight', 'mp_per_g', 'efg_pct', 'ft_pct']
RMSE value: 1.980091369959789

Model: KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=10, p=2,
          weights='uniform')
Best features: ['height', 'weight', 'mp_per_g', 'efg_pct', 'ft_pct']
RMSE value: 2.0947238522243294



**Regression Ensembler:**

We will utilize the `RegressionEnsembler` class that Professor Sun provided to us and see if we can get a better result combining two models with the lowest rmse values.

In [6]:
class RegressionEnsembler(BaseEstimator, RegressorMixin):
    def __init__(self, estimators, learn_weights=True):
        self.estimators = estimators
        self.learn_weights = learn_weights
        
    def fit(self, X, y):
        # check that X and y have the correct shape
        X, y = check_X_y(X, y)
        
        # store the training features and the labels
        self.X_ = X
        self.y_ = y
        
        # call the fit method of each of the estimators
        for estimator in self.estimators:
            estimator.fit(X, y)
            
        # if we wish to learn the "optimal" weights from the training data
        if self.learn_weights:
            # get prediction from each estimator on the training data
            predictions = []
            for estimator in self.estimators:
                predictions.append(estimator.predict(X))
            Y_ = np.column_stack(predictions)
        
            # fit linear regression on top of the estimators' predictions
            self.ensembler = LinearRegression(fit_intercept=False)
            self.ensembler.fit(Y_, y)
            
        return self
    
    def predict(self, X):
        # check that fit has been called
        check_is_fitted(self, ['X_', 'y_'])
        
        # check that X has the right form
        X = check_array(X)
        
        # calculate predictions from the estimators
        predictions = []
        for estimator in self.estimators:
            predictions.append(estimator.predict(X))
        Y_ = np.column_stack(predictions)
        
        # return predictions
        if self.learn_weights:
            return self.ensembler.predict(Y_)
        else:
            return Y_.mean(axis=1)

In [8]:
# Evaluate the ensembler model with the best set of features and lowest RMSE
model = RegressionEnsembler([RandomForestRegressor(n_estimators = 10), 
                             KNeighborsRegressor(n_neighbors = 10)])
dictionary = find_best_features(model, list_of_features, y)
min_rmse = min(dictionary.items(), key=lambda x: x[1]) 
print("Model: " + str(model))
print("Best features: " + min_rmse[0])
print("RMSE value: " + str(min_rmse[1]) + "\n")

Model: RegressionEnsembler(estimators=[RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estima...wski',
          metric_params=None, n_jobs=None, n_neighbors=10, p=2,
          weights='uniform')],
          learn_weights=True)
Best features: ['name', 'height', 'weight', 'pos', 'mp_per_g', 'efg_pct', 'ft_pct']
RMSE value: 1.9588425132406198



It appears that the `RegressionEnsembler` model with `['name', 'height', 'weight', 'pos', 'mp_per_g', 'efg_pct', 'ft_pct']` as the features yields the lowest `rmse` of `~1.959`.

### Hyperparameter Tuning:

Using the best model and set of features, we will now determine the best scaler methods. Then, we will determine the best hyperparameters for the `RandomForestRegressor` model.

In [17]:
list_of_scaler = [StandardScaler(), Normalizer(), MinMaxScaler(), RobustScaler()]
best_features = ['name', 'height', 'weight', 'pos', 'mp_per_g', 'efg_pct', 'ft_pct']
# Function that determines the best scaler method
def find_best_scaler(model, list_of_scaler, y):
    # Set up the pipeline
    
    # Create a dictionary of features and its respective rmse value
    dictionary = {}
    for scaler in list_of_scaler:
        pipeline = Pipeline([("vectorizer", vec), ("scaler", scaler), ("fit", model)])
        X_dict = player_df[best_features].to_dict(orient="records")
        scores = cross_val_score(pipeline, X_dict, y, cv=10, scoring="neg_mean_squared_error")
        rmse = np.sqrt(np.mean(-scores))
        dictionary.update({str(scaler):rmse})
    return dictionary

In [8]:
model = RegressionEnsembler([RandomForestRegressor(n_estimators = 10), 
                             KNeighborsRegressor(n_neighbors = 10)])
dictionary = find_best_scaler(model, list_of_scaler, y)
min_rmse = min(dictionary.items(), key=lambda x: x[1]) 
print("Model: " + str(model))
print("Best scaler: " + min_rmse[0])
print("RMSE value: " + str(min_rmse[1]) + "\n")

Model: RegressionEnsembler(estimators=[RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estima...wski',
          metric_params=None, n_jobs=None, n_neighbors=10, p=2,
          weights='uniform')],
          learn_weights=True)
Best scaler: StandardScaler(copy=True, with_mean=True, with_std=True)
RMSE value: 2.00823719083



### Train Error vs. Validation Error:

We will determine what the best value for `n_estimators` is by calculating the training error and validation error using the `RobustScaler`, `best_features`.

In [13]:
vec = DictVectorizer(sparse=False)
scaler = StandardScaler()

# Calculates estimate of test error based on 10-fold cross validation
def val_error(k):
    model = RegressionEnsembler([RandomForestRegressor(n_estimators = k), 
                                 KNeighborsRegressor(n_neighbors = 10)])
    pipeline = Pipeline([("vectorizer", vec), ("scaler", scaler), ("fit", model)])
    X_dict = player_df[best_features].to_dict(orient="records")
    scores = cross_val_score(pipeline, X_dict, y, cv=10, scoring="neg_mean_squared_error")
    rmse = np.sqrt(np.mean(-scores))
    return rmse

# Calculates the train error based on 10-fold cross validation
def train_error(k):
    X_dict = player_df[best_features].to_dict(orient="records")
    vec.fit(X_dict)
    X_train = vec.transform(X_dict)
    scaler.fit(X_train)
    X_train_sc = scaler.transform(X_train)
    
    model = RegressionEnsembler([RandomForestRegressor(n_estimators = k), 
                                 KNeighborsRegressor(n_neighbors = 10)])
    model.fit(X_train_sc, y)

    y_train_pred = model.predict(X_train_sc)
    rmse = np.sqrt(((y - y_train_pred) ** 2).mean())
    return rmse

In [ ]:
# Plot train and validation error
ks = pd.Series(range(1,50))
ks.index = range(1,50)
test_errs = ks.apply(lambda m: val_error(m))
test_errs.plot.line()
train_errs = ks.apply(lambda m: train_error(m))
train_errs.plot.line(title = "Train vs. Validation")
print("Best value for n_estimators: " + str(test_errs.sort_values().idxmin()))

### Predicting average points per game in 2017:

In [18]:
unknown_df = test_df

In [19]:
unknown_df["pts_per_g"] = np.nan
unknown_df.head()

,name,season,age,team_id,pos,g,gs,mp_per_g,fg_pct,fg3_per_g,...,pf_per_g,pts_per_g,height,weight,wins,losses,rank_team,division,conference,finish
0,Alex Abrines,2017.0,24.0,OKC,SG,75.0,8.0,15.1,0.395,1.1,...,1.7,NaN,78,200,48.0,34.0,2.0,Northwest,W,5.0
1,Steven Adams,2017.0,24.0,OKC,C,76.0,76.0,32.7,0.629,0.0,...,2.8,NaN,84,265,48.0,34.0,2.0,Northwest,W,5.0
2,Carmelo Anthony,2017.0,33.0,OKC,PF,78.0,78.0,32.1,0.404,2.2,...,2.5,NaN,80,240,48.0,34.0,2.0,Northwest,W,5.0
3,Corey Brewer,2017.0,31.0,OKC,SG,18.0,16.0,28.6,0.444,1.3,...,3.1,NaN,81,186,48.0,34.0,2.0,Northwest,W,5.0
4,Nick Collison,2017.0,37.0,OKC,PF,15.0,0.0,5.0,0.684,0.0,...,0.5,NaN,82,255,48.0,34.0,2.0,Northwest,W,5.0


In [20]:
X_dict = player_df[best_features].to_dict(orient="records")
X_new_dict = unknown_df[best_features].to_dict(orient="records")
y = player_df["pts_per_g"]

scaler = StandardScaler()
model = RegressionEnsembler([RandomForestRegressor(n_estimators = 10), 
                                     KNeighborsRegressor(n_neighbors = 10)])

vec.fit(X_dict)
X_train = vec.transform(X_dict)
X_new = vec.transform(X_new_dict)

scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_new_sc = scaler.transform(X_new)

model.fit(X_train_sc, y)

unknown_df["pts_per_g"] = model.predict(X_new_sc)

**Predicted average points per game for the top 5 shooters in 2017:**

In [23]:
unknown_df[["name", "pos", "pts_per_g"]].sort_values("pts_per_g", ascending=False).head()

,name,pos,pts_per_g
444,Kevin Durant,PF,25.936307
593,LeBron James,PF,25.743659
162,Damian Lillard,PG,25.516807
134,DeMarcus Cousins,C,25.199666
202,James Harden,SG,25.180160


**Actual average points per game for top 5 shooters in 2017:**

In [25]:
test_df = final_df[final_df["season"] == 2017].reset_index().drop(columns=["index"])
test_df[["name", "pos", "pts_per_g"]].sort_values("pts_per_g", ascending=False).head()

,name,pos,pts_per_g
202,James Harden,SG,30.4
137,Anthony Davis,PF,28.1
593,LeBron James,PF,27.5
236,Giannis Antetokounmpo,PF,26.9
162,Damian Lillard,PG,26.9
